In [1]:
from pymongo import MongoClient
from bs4 import BeautifulSoup
import pandas as pd

we_eat_client = MongoClient()
we_eat_db = we_eat_client['we_eat']
website_collection = we_eat_db['websites']
more_business_info_collection = we_eat_db['more_business_info']

In [2]:
website_collection.count_documents({})

657

In [3]:
a = website_collection.find()[:3]

In [5]:
def parse_bizinfo(bizinfos, alias):
    biz = {}
    biz['alias'] = alias
    for bizinfo in bizinfos:
        dt = bizinfo.select_one('dt')
        cat_name = dt.text.strip()
        dd = bizinfo.select_one('dd')
        response = dd.text.strip()
        biz[cat_name] = response
    return biz

In [6]:
def save_bizinfo(bizinfo_data):
    more_business_info_collection.delete_many({
        'alias' : bizinfo_data['alias']
    })
    more_business_info_collection.insert_one(bizinfo_data)

In [7]:
def collect_bizinfo(website_collection):
    for website in website_collection.find():
        url = website['url']
        alias = url.rpartition('/')[2]
        html = website['html']
        soup = BeautifulSoup(html, 'html.parser')
        bizinfos = soup.select('div.short-def-list dl')
        print(alias)
        data = parse_bizinfo(bizinfos, alias)
        save_bizinfo(data)

In [4]:
for website in a:
    html = website['html']
    soup = BeautifulSoup(html, 'html.parser')
    bizinfo = soup.select('div.short-def-list dl')
    data = get_biz_info(bizinfo)
    

NameError: name 'get_biz_info' is not defined

In [8]:
website = website_collection.find()[2:3]

In [9]:
html = website[0]['html']

In [10]:
soup = BeautifulSoup(html, 'html.parser')

In [11]:
bizinfo = soup.select('div.short-def-list dl')

In [12]:
bizinfo[0].find_all('dt')

[<dt class="attribute-key">
                             Liked by Vegans
                         </dt>]

In [13]:
len(bizinfo)

26

In [14]:
get_biz_info(bizinfo)

NameError: name 'get_biz_info' is not defined

In [ ]:
%pdb

In [17]:
#collect_bizinfo(website_collection)

In [16]:
more_business_info_collection.count_documents({})

657

In [18]:
more_business_info_collection.find_one({'alias' : 'nachyo-average-seattle'})

{'Delivery': 'No',
 'Good for Kids': 'Yes',
 'Take-out': 'Yes',
 'Takes Reservations': 'No',
 '_id': ObjectId('5c0841f291d56fd487300588'),
 'alias': 'nachyo-average-seattle'}

In [19]:
more_business_info_collection.delete_many({'alias': {'$exists': False}})

In [20]:
biz_dicts = pd.DataFrame(list(more_business_info_collection.find()))

In [21]:
cols = list(biz_dicts.columns)

In [22]:
sorted_cols = cols[-2:] + cols[:-2]

In [23]:
biz_dicts = biz_dicts[sorted_cols]

In [24]:
biz_dicts.columns

Index(['_id', 'alias', 'Accepts Apple Pay', 'Accepts Bitcoin',
       'Accepts Credit Cards', 'Accepts Google Pay', 'Alcohol', 'Ambience',
       'Attire', 'Best Nights', 'Bike Parking', 'Cannabis Products', 'Caters',
       'Coat Check', 'Delivery', 'Dogs Allowed', 'Drive-Thru',
       'Gender Neutral Restrooms', 'Good For', 'Good For Dancing',
       'Good for Groups', 'Good for Kids', 'Good for Working', 'Happy Hour',
       'Has Dairy-free Options', 'Has Gluten-free Options',
       'Has Halal Options', 'Has Kosher Options', 'Has Pool Table',
       'Has Soy-free Options', 'Has TV', 'Liked by Vegans',
       'Liked by Vegetarians', 'Music', 'Noise Level',
       'Offers Military Discount', 'Open to All', 'Outdoor Seating', 'Parking',
       'Smoking', 'Take-out', 'Takes Reservations', 'Waiter Service',
       'Wheelchair Accessible', 'Wi-Fi'],
      dtype='object')

In [25]:
bizinfo_df = biz_dicts.drop(['_id'], axis=1).set_index('alias')

In [ ]:
bizinfo_df.to_pickle('bizinfo_df.pkl')

In [29]:
bizinfo_df.head()

,Accepts Apple Pay,Accepts Bitcoin,Accepts Credit Cards,Accepts Google Pay,Alcohol,Ambience,Attire,Best Nights,Bike Parking,Cannabis Products,...,Offers Military Discount,Open to All,Outdoor Seating,Parking,Smoking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi
alias,,,,,,,,,,,,,,,,,,,,,
red-cedar-and-sage-seattle,Yes,NaN,Yes,No,Full Bar,Casual,Casual,"Thu, Fri, Sat",Yes,NaN,...,Yes,NaN,Yes,Street,No,Yes,Yes,Yes,Yes,Free
noi-thai-cuisine-seattle-3,Yes,NaN,Yes,No,Full Bar,"Casual, Romantic, Classy",Casual,NaN,Yes,NaN,...,NaN,NaN,Yes,NaN,NaN,Yes,Yes,Yes,Yes,No
congeez-seattle,Yes,No,Yes,Yes,No,NaN,Casual,NaN,Yes,NaN,...,NaN,Yes,No,"Street, Private Lot",NaN,Yes,No,NaN,Yes,No
the-crab-pot-seattle-2,Yes,NaN,Yes,Yes,Full Bar,Touristy,Casual,NaN,Yes,NaN,...,NaN,NaN,Yes,Street,NaN,Yes,No,Yes,NaN,Free
musashis-seattle-3,No,NaN,Yes,NaN,Beer & Wine Only,Casual,Casual,NaN,Yes,NaN,...,NaN,NaN,No,Street,NaN,Yes,No,Yes,NaN,No
